In [85]:
### Benjamin Tollison ###
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from decimal import Decimal
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

### Setting the Globals based off of my information

In [86]:
astro_constants = pd.read_csv('AstroConstants.csv')
ephemeris = pd.read_csv('EphemerisData.csv')
x = 20
y = 4
earth_parking_radius = 10*x + 450 + astro_constants['Earth'][1]   # km
mars_periapsis_radius = 15*y +150 + astro_constants['Mars'][1]
mars_apoapsis_radius = 275*y + 2500 + astro_constants['Mars'][1]
def GetIndexFromList(list, target_value):
    for key, value in enumerate(list):
        if value == target_value:
            return key
departure_index = GetIndexFromList(ephemeris['Epoch'],'10-Nov-26')
departure_data = {key:value[departure_index] for key,value in ephemeris.items()}
mu_sun = astro_constants['Sun'][0]
mu_earth = astro_constants['Earth'][0]
mu_mars = astro_constants['Mars'][0]
unit_length_conversion = astro_constants['Earth'][3]
unit_time_conversion = np.sqrt(unit_length_conversion**3/mu_sun) # sec
print(unit_length_conversion,'km')
print(unit_time_conversion,'seconds')
print(unit_time_conversion/(60*60*24),'days')
print(mu_sun,unit_length_conversion**3/unit_time_conversion**2)
display(departure_data)

149597870.7 km
5022642.890913032 seconds
58.132440867048985 days
132712440041.9393 132712440041.93929


{'Epoch': '10-Nov-26',
 'Earth X': 100622923.6,
 'Earth Y': 99795464.56,
 'Earth Z': 43259105.48,
 'Earth Xdot': -22.36016069,
 'Earth Ydot': 18.46384229,
 'Earth Zdot': 8.003854305,
 'Mars X': -60800396.04,
 'Mars Y': 210334752.7,
 'Mars Z': 98115468.4,
 'Mars Xdot': -22.52065467,
 'Mars Ydot': -3.932675694,
 'Mars Zdot': -1.196445272}

In [102]:
def OneATransfer(a0,s,c,non_dimensional_TOF):
  alpha = lambda a: 2*np.arcsin(np.sqrt(s/(2*a)))
  beta = lambda a: 2*np.arcsin(np.sqrt((s-c)/(2*a)))
  LambertFunction = lambda a: np.sqrt(a**3)*(alpha(a)-beta(a) - (np.sin(alpha(a))-np.sin(beta(a)))) - non_dimensional_TOF
  iteration_max = 1000
  return print('TODO 1A')
def OneBTransfer(a0,s,c,non_dimensional_TOF):
  alpha = lambda a: 2*np.pi - 2*np.arcsin(np.sqrt(s/(2*a)))
  beta = lambda a: 2*np.arcsin(np.sqrt((s-c)/(2*a)))
  return print('TODO 1B')
def TwoATransfer(a0,s,c,non_dimensional_TOF):
  alpha = lambda a: 2*np.arcsin(np.sqrt(s/(2*a)))
  beta = lambda a: -2*np.arcsin(np.sqrt((s-c)/(2*a)))
  return print('TODO 2A')
def TwoBTransfer(a0,s,c,non_dimensional_TOF):
  alpha = lambda a: 2*np.arcsin(np.sqrt(s/(2*a)))
  beta = lambda a: 2*np.arcsin(np.sqrt((s-c)/(2*a)))
  return print('TODO 2B')
def EllipticTransfer(s,c,non_dimensional_TOF,TOF_p1,TOF_p2):
  a_min = 0.5*s
  alpha = lambda a: 2*np.arcsin(np.sqrt(s/(2*a)))
  beta = lambda a: 2*np.arcsin(np.sqrt((s-c)/(2*a)))
  TOFm1 = np.sqrt(a_min**3/1)*((alpha(a_min) - np.sin(alpha(a_min))) - (beta(a_min)-np.sin(beta(a_min))))
  TOFm2 = np.sqrt(a_min**3/1)*((alpha(a_min) - np.sin(alpha(a_min))) + (beta(a_min)-np.sin(beta(a_min))))
  a0 = a_min + 1e-4
  if TOF_p1 < non_dimensional_TOF and non_dimensional_TOF < TOFm1:
    OneATransfer(a0,s,c,non_dimensional_TOF)
  if TOFm1 < non_dimensional_TOF:
    OneBTransfer(a0,s,c,non_dimensional_TOF)
  if TOF_p2 < non_dimensional_TOF and non_dimensional_TOF < TOFm2:
    TwoATransfer(a0,s,c,non_dimensional_TOF)
  if TOFm2 < non_dimensional_TOF:
    TwoBTransfer(a0,s,c,non_dimensional_TOF)
  return print('TODO Elliptic')
def HyperbolicTransfer():
  return print('TODO Hyperbolic')
def LamberSolverA(TOF):
  arrival_index = departure_index + TOF
  arrival_data = {key:value[arrival_index] for key,value in ephemeris.items()}
  r1_vec = np.array([departure_data['Earth X'],departure_data['Earth Y'],departure_data['Earth Z']])/unit_length_conversion
  r2_vec = np.array([arrival_data['Earth X'],arrival_data['Earth Y'],arrival_data['Earth Z']])/unit_length_conversion
  c_vec = r2_vec - r1_vec
  r1,r2,c = np.linalg.norm(r1_vec),np.linalg.norm(r2_vec),np.linalg.norm(c_vec)
  s = 0.5*(r1+r2+c)
  non_dimensional_TOF = (TOF*24*3600)/unit_time_conversion
  TOF_p1 = (1/3)*np.sqrt(2/1)*(s**(3/2) - (s-c)**(3/2))
  TOF_p2 = (1/3)*np.sqrt(2/1)*(s**(3/2) + (s-c)**(3/2))
  if non_dimensional_TOF < TOF_p1 or non_dimensional_TOF <= TOF_p2:
    HyperbolicTransfer()
  if non_dimensional_TOF > TOF_p2:
    EllipticTransfer(s,c,non_dimensional_TOF,TOF_p1,TOF_p2)
    return print('TODO LamberSolverA')
LamberSolverA(182)

TODO 1A
TODO 2A
TODO Elliptic
TODO LamberSolverA
